In [ ]:
import zipfile
import os

# Path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/brain_tumor_dataset1.zip'

# Destination folder to extract
extract_path = '/content/dataset'

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Unzipping complete!")


Unzipping complete!


In [ ]:
import zipfile
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from glob import glob
# STEP 1: Imports
import os
import numpy as np
import shap
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import zipfile
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import os

In [ ]:


def segment_and_save_masks_cpu(image_path, output_dir, num_segments=10):
    # Get image name without extension
    img_name = os.path.splitext(os.path.basename(image_path))[0]

    # Load image in grayscale
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Warning: Could not read {image_path}")
        return

    h, w = img.shape
    flat_img = img.reshape((-1, 1)).astype(np.float32)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=num_segments, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(flat_img)

    # Create and save masks
    for i in range(num_segments):
        mask = (labels == i).astype(np.uint8).reshape((h, w)) * 255
        mask_filename = f"{img_name}_segment_{i+1}.png"
        mask_path = os.path.join(output_dir, mask_filename)
        cv2.imwrite(mask_path, mask)

def batch_segment_directory_cpu(input_dir, output_dir='segmentation_masks', num_segments=10):
    # Create output directory if not exists
    os.makedirs(output_dir, exist_ok=True)

    # Collect all image files
    image_paths = glob.glob(os.path.join(input_dir, "*.png")) + \
                  glob.glob(os.path.join(input_dir, "*.jpg")) + \
                  glob.glob(os.path.join(input_dir, "*.jpeg"))

    total = len(image_paths)
    print(f"Found {total} images. Starting segmentation...")

    # Process each image
    for idx, image_path in enumerate(image_paths, 1):
        print(f"[{idx}/{total}] Processing: {os.path.basename(image_path)}")
        segment_and_save_masks_cpu(image_path, output_dir, num_segments=num_segments)

    print("✅ All images processed and masks saved to:", output_dir)

# Example usage:
batch_segment_directory_cpu('/content/dataset/brain_tumor_dataset1', output_dir='all_masks_output', num_segments=10)


Found 179 images. Starting segmentation...
[1/179] Processing: no 9.png
[2/179] Processing: Y92.png
[3/179] Processing: Y14.jpg
[4/179] Processing: 33 no.jpg
[5/179] Processing: 46 no.jpg
[6/179] Processing: 6 no.jpg
[7/179] Processing: Y154.jpg
[8/179] Processing: No16.jpg
[9/179] Processing: 12 no.jpg
[10/179] Processing: Y186.jpg
[11/179] Processing: Y153.jpg
[12/179] Processing: Y79.jpg
[13/179] Processing: 11 no.jpg
[14/179] Processing: 36 no.jpg
[15/179] Processing: Y78.jpg
[16/179] Processing: No13.jpg
[17/179] Processing: Y95.jpg
[18/179] Processing: Y70.jpg
[19/179] Processing: 3 no.jpg
[20/179] Processing: no 94.jpg
[21/179] Processing: Y46.jpg
[22/179] Processing: Y77.jpg
[23/179] Processing: 34 no.jpg
[24/179] Processing: Y245.jpg
[25/179] Processing: Y168.jpg
[26/179] Processing: no 1.jpg
[27/179] Processing: Y81.jpg
[28/179] Processing: 28 no.jpg
[29/179] Processing: no 6.jpg
[30/179] Processing: Y82.jpg
[31/179] Processing: Y28.jpg
[32/179] Processing: Y188.jpg
[33/179] 

In [ ]:
import shutil

def zip_output_folder(folder_path, zip_name='segmentation_masks.zip'):
    shutil.make_archive(base_name=zip_name.replace('.zip', ''), format='zip', root_dir=folder_path)
    print(f"✅ Zipped folder created: {zip_name}")

# Example usage:
zip_output_folder('/content/all_masks_output')


✅ Zipped folder created: segmentation_masks.zip


In [ ]:
# Define paths
concepts_base_path = "/content/dataset1234/kmeans/concept"
background_path = "/content/dataset1234/kmeans/random"
model_path = "/content/drive/MyDrive/new_mri.keras"
intermediate_layer_name = "bottleneck"  # Change if need

In [ ]:
# Load model and create bottleneck model
model = load_model(model_path)
bottleneck_output = model.get_layer(intermediate_layer_name).output
bottleneck_model = Model(inputs=model.input, outputs=bottleneck_output)

In [ ]:
# Helper function to load images
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def load_images_from_folder(folder_path, target_size=(224, 224), color_mode='grayscale', max_images=None):
    images = []
    filenames = os.listdir(folder_path)
    for i, file in enumerate(filenames):
        if max_images and i >= max_images:
            break
        img_path = os.path.join(folder_path, file)
        img = load_img(img_path, target_size=target_size, color_mode=color_mode)
        img = img_to_array(img)
        images.append(img)
    return np.array(images)

In [ ]:
# Loop through all concept folders
concept_scores = {}
for concept_name in os.listdir(concepts_base_path):
    concept_folder = os.path.join(concepts_base_path, concept_name)
    if not os.path.isdir(concept_folder):
        continue
    concept_images = load_images_from_folder(concept_folder)
    concept_features = bottleneck_model.predict(concept_images)

    # Compute ConceptSHAP score (cosine similarity)
    similarities = cosine_similarity(concept_features, background_features)

    mean_score = np.mean(similarities)
    concept_scores[concept_name] = mean_score
    print(f"✅ ConceptSHAP score for '{concept_name}': {mean_score:.4f}")


In [ ]:
# Plot
plt.figure(figsize=(10, 5))
plt.bar(concept_scores.keys(), concept_scores.values(), color='skyblue')
plt.title("ConceptSHAP Scores")
plt.xlabel("Concept")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()
